# Sakamoto BERT

## Preprocess

install Transformers

In [ ]:
!pip install transformers

download the BERT Japanese pre-trained model

In [ ]:
!wget http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip

In [ ]:
!unzip Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip -d bert

copy Sakamoto SST-2

In [ ]:
!cp -r drive/My\ Drive/sakamoto-bert/SST-2 sakamoto_sst2

## Training

clone the repository

In [ ]:
!git clone https://github.com/huggingface/transformers

In [ ]:
%cd transformers
!git checkout bb9559a7f98fc18747fd957c3bd2a6e4c1111e45
%cd ..

fine-tune the model

In [ ]:
!python transformers/examples/text-classification/run_glue.py \
    --model_name_or_path bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers \
    --task_name SST-2 \
    --do_train \
    --do_eval \
    --data_dir sakamoto_sst2 \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir bert_sakamoto_sst2

copy the model

In [ ]:
!cp -r bert_sakamoto_sst2 drive/My\ Drive/sakamoto-bert/model

## Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert_sakamoto_sst2')
model = AutoModelForSequenceClassification.from_pretrained('bert_sakamoto_sst2', return_dict=True)

In [ ]:
classes = ['not sksk_sskn', 'is sksk_sskn']

In [ ]:
import csv

sequences = []
with open('sakamoto_sst2/test.tsv', newline='') as f:
    reader = csv.DictReader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        sequences.append(row['sentence'])

In [ ]:
for sequence in sequences:
    sequence_tensor = tokenizer(sequence, return_tensors='pt')
    classification_logits = model(**sequence_tensor).logits
    results = torch.softmax(classification_logits, dim=1).tolist()[0]

    print(sequence)
    for i in range(len(classes)):
        print(f'{classes[i]}: {results[i] * 100}%')
    print()

# References

Transformers

- [transformers/examples/text-classification at master · huggingface/transformers](https://github.com/huggingface/transformers/tree/master/examples/text-classification)
- [Summary of the tasks — transformers 3.0.2 documentation](https://huggingface.co/transformers/task_summary.html#sequence-classification)

BERT Japanese Pretrained Model

- [BERT日本語Pretrainedモデル - KUROHASHI-CHU-MURAWAKI LAB](http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT%E6%97%A5%E6%9C%AC%E8%AA%9EPretrained%E3%83%A2%E3%83%87%E3%83%AB)

The Stanford Sentiment Treebank	

- [GLUE Benchmark](https://gluebenchmark.com/tasks)